In [2]:
import argparse
import data.ts_datasets as my
import pandas as pd
from models import DumbMLP4TS, ARv2
import torch
import numpy as np
import pickle as pkl
import time 
import sys

In [2]:
DEVICE = torch.device("cpu")
dataset="ETTh1_96"
batch_size=64
multi_channel=False

file_dict = {
    "ETTh1": "ETT-small/ETTh1.csv",
    "ETTh2": "ETT-small/ETTh2.csv",
    "ETTm1": "ETT-small/ETTm1.csv",
    "ETTm2": "ETT-small/ETTm2.csv",
    "ECL": "electricity/electricity.csv",
    "ER": "exchange_rate/exchange_rate.csv",
    "ILI": "illness/national_illness.csv",
    "Traffic": "traffic/traffic.csv",
    "Weather": "weather/weather.csv"
}


params = dataset.split("_")
prefix = params[0]
horizon = int(params[1])
assert prefix in file_dict, f"Invalid dataset {dataset} from possible {list(file_dict.keys())}"
input_length = 96 if prefix == "ILI" else 512
train_loader, val_loader, test_loader = my.get_timeseries_dataloaders(
    f"./data/ts_datasets/all_six_datasets/{file_dict[prefix]}", 
    batch_size=batch_size, 
    seq_len=input_length,
    forecast_horizon=horizon,
    multi_channel=multi_channel
)
train_data, val_data, test_data = train_loader.dataset, val_loader.dataset, test_loader.dataset
train_df, val_df, test_df = pd.DataFrame(train_data.data), pd.DataFrame(val_data.data), pd.DataFrame(test_data.data)

do_diff = True
start_time = time.perf_counter()
ar = ARv2(input_length, horizon)
ar.fit_raw(train_df, val_df, DEVICE, 10, do_diff, use_ols=True, first_lag=400)
print(f"AR search for {dataset} {horizon} finished in {time.perf_counter() - start_time} seconds)")
ar.fit_preset(pd.concat((train_df, val_df)), ar.best_lags, do_diff, torch.device("cpu"), use_ols=True)
mse, mae = ar.test_loss_acc_df(test_df, torch.device("cpu"))
print(f"Final AR metrics for {dataset} {horizon}: MSE {np.mean(mse)} | MAE {np.mean(mae)}")

10 hours limit


  0%|          | 0/112 [00:00<?, ?it/s]

400


  1%|          | 1/112 [00:00<01:02,  1.76it/s]

401


...

510


 99%|█████████▉| 111/112 [01:15<00:00,  1.34it/s]

511


100%|██████████| 112/112 [01:15<00:00,  1.48it/s]


Best Lags: 503
AR search for ETTh1_96 96 finished in 75.8529167752713 seconds)
Selected Lags: 503
Final AR metrics for ETTh1_96 96: MSE 0.3579343259334564 | MAE 0.38869863748550415


In [19]:
for i, x in enumerate(test_loader):
    if i == 0:
        print(x[1].shape)
    pass

print(i)

torch.Size([64, 1, 96])
304


In [11]:
list1 = np.array([1, 2, 3, 4, 5])
list2 = np.array([5, 6, 7, 8, 9])
y = np.array([1.5, 2.5, 3.5, 4.5, 5.5])

# find alpha such that alpha * list1 + (1 - alpha) * list2 is closest to y
alpha = np.sum(-2*(list2-y)*(list1-list2)) / np.sum(2*(list1-list2)**2)
print(alpha)
print(alpha * list1 + (1 - alpha) * list2)

0.875
[1.5 2.5 3.5 4.5 5.5]


In [4]:
a = np.array([1, 2, 3, 4, 5])
print(np.flip(a, 0))

[5 4 3 2 1]
